Anushna Prakash  
DATA 598 - Deep Learning  
January 28 2022  
# <center> Homework 3 </center>  
## Part I: The Effect of BatchNorm on a ConvNet  

First, download and preprocess the data.

In [1]:
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt

# download dataset (~117M in size)
train_dataset = FashionMNIST('./data', train=True, download=True)
X_train = train_dataset.data # torch tensor of type uint8
y_train = train_dataset.targets # torch tensor of type Long
test_dataset = FashionMNIST('./data', train=False, download=True)
X_test = test_dataset.data
y_test = test_dataset.targets

# choose a subsample of 10% of the data:
idxs_train = torch.from_numpy(
    np.random.choice(X_train.shape[0], replace=False, size=X_train.shape[0]//10)).long()
X_train, y_train = X_train[idxs_train], y_train[idxs_train]
# idxs_test = torch.from_numpy(
#     np.random.choice(X_test.shape[0], replace=False, size=X_test.shape[0]//10))
# X_test, y_test = X_test[idxs_test], y_test[idxs_test]

print(f'X_train.shape = {X_train.shape}')
print(f'n_train: {X_train.shape[0]}, n_test: {X_test.shape[0]}')
print(f'Image size: {X_train.shape[1:]}')

f, ax = plt.subplots(1, 5, figsize=(20, 4))
for i, idx in enumerate(np.random.choice(X_train.shape[0], 5)):
    ax[i].imshow(X_train[idx], cmap='gray', vmin=0, vmax=255)
    ax[i].set_title(f'Label = {y_train[idx]}', fontsize=20)
    
# Normalize dataset: pixel values lie between 0 and 255
# Normalize them so the pixelwise mean is zero and standard deviation is 1

X_train = X_train.float()  # convert to float32
X_train = X_train.view(-1, 784)
mean, std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean[None, :]) / (std[None, :] + 1e-6)  # avoid divide by zero

X_test = X_test.float()
X_test = X_test.view(-1, 784)
X_test = (X_test - mean[None, :]) / (std[None, :] + 1e-6)

99.7%

100.6%


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



85.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Create a Convolutional Neural Net with two convolutional layers:  
Input $(1, 28, 28) \rightarrow$ Convolution with $k=5$, `filters=16`, `padding=2` $\rightarrow$ ReLU $\rightarrow$ MaxPool with $k=2 \rightarrow$ Convolution with $k=5$, `filters=32`, `padding=2` $\rightarrow$ ReLU $\rightarrow$ MaxPool with $k=2 \rightarrow$ Linear output = 10

### Step 1: Figure out size `S` to flatten to  
This is for the `torch.nn.Linear` final step.  

In [30]:
image_size = 228
# 1 batch, 1 channel, 228x228 image
dummy = torch.randn(1, 1, image_size, image_size)

# Run dummy thru input layers
conv1 = torch.nn.Conv2d(
    in_channels=1, out_channels=16, kernel_size=5,
    padding=2, stride=1
)
out = conv1(dummy)
# 1 batch, 16 channels, image size, image size
print(out.shape)
relu = torch.nn.ReLU()
out = relu(out)
# 1 batch, 16 channels, image size, image size
print(out.shape)
pool = torch.nn.MaxPool2d(2)
out = pool(out)
# 1 batch, 16 channels, image size / kernel input size, image size / kernel input size
print(out.shape)

# Run thru second part
# Run dummy thru input layers
# Now has 16 input channels
conv2 = torch.nn.Conv2d(
    in_channels=16, out_channels=32, kernel_size=5,
    padding=2, stride=1
)
out = conv2(out)
# 1 batch, 32 channels, image size, image size
print(out.shape)
relu = torch.nn.ReLU()
out = relu(out)
# 1 batch, 32 channels, image size, image size
print(out.shape)
pool = torch.nn.MaxPool2d(2)
out = pool(out)
# 1 batch, 32 channels, image size / kernel input size, image size / kernel input size
print(out.shape)

# Now convert final out shape into 1 x 10???

torch.Size([1, 16, 228, 228])
torch.Size([1, 16, 228, 228])
torch.Size([1, 16, 114, 114])
torch.Size([1, 32, 114, 114])
torch.Size([1, 32, 114, 114])
torch.Size([1, 32, 57, 57])


In [8]:
import torch
class ConvNet(torch.nn.Module):
    """
    Defines a convolution neural net with 2 convolution layers
    Does not employ batch norming
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv_ensemble_1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        self.conv_ensemble_2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        # Final layer output is batch x channels x imagesize / 4 x imagesize / 4
        self.fully_connected_layer = torch.nn.Linear(7*7*32, num_classes)
    
    def forward(self, x):
        x = x.view(-1, 1, 28, 28) # Resize, needs channel
        out = self.conv_ensemble_1(x) # Run thru layer 1
        out = self.conv_ensemble_2(out) # Run thru layer 2
        out = out.view(out.shape[0], -1)  # flatten output
        out = self.fully_connected_layer(out)  # output layer
        return out

Create a Convolutional Neural Net that is the same as above, but has batch norm applied after each `MaxPool2d` layer.

In [9]:
class BatchNormConvNet(torch.nn.Module):
    """
    Defines a convolution neural net with 2 convolution layers
    Employ batch norming
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv_ensemble_1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
            torch.nn.BatchNorm2d(num_features=16))
        self.conv_ensemble_2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
            torch.nn.BatchNorm2d(num_features=32))
        self.fully_connected_layer = torch.nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28) # Resize, needs channel
        out = self.conv_ensemble_1(x) # Run thru layer 1
        out = self.conv_ensemble_2(out) # Run thru layer 2
        out = out.view(out.shape[0], -1)  # flatten output
        out = self.fully_connected_layer(out)  # output layer
        return out

In [31]:
# Test
model = ConvNet(num_classes=10)
model2 = BatchNormConvNet(num_classes=10)